In [1]:
import cv2
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
########################################### Select Image Pair

IMAGE_PAIR = "CONES"     ### Choices:     CONES     DOLLS     MOEBIUS     ROCKS     CLOTH

########################################################################################################

if IMAGE_PAIR == "CONES":
    img_left = cv2.imread("./Stereo_3D_Reconstruction/cones1.png", cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread("./Stereo_3D_Reconstruction/cones2.png", cv2.IMREAD_GRAYSCALE)
    img_left_color = cv2.cvtColor(cv2.imread("./Stereo_3D_Reconstruction/cones1.png"), cv2.COLOR_BGR2RGB) # BGR
    point_cloud_cluster = 20
    focal_length_1, focal_length_2 = 360.0, 360.0
    cx, cy = 225, 188
    NNDR_THRESHOLD = 0.9
    
elif IMAGE_PAIR =="DOLLS":
    img_left = cv2.imread("./Stereo_3D_Reconstruction/dolls1.png", cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread("./Stereo_3D_Reconstruction/dolls2.png", cv2.IMREAD_GRAYSCALE)
    img_left_color = cv2.cvtColor(cv2.imread("./Stereo_3D_Reconstruction/dolls1.png"), cv2.COLOR_BGR2RGB) # BGR
    point_cloud_cluster = 300
    focal_length_1, focal_length_2 = 1512.0, 1512.0
    cx, cy = 695, 555
    NNDR_THRESHOLD = 0.85
    
elif IMAGE_PAIR =="MOEBIUS":
    img_left = cv2.imread("./Stereo_3D_Reconstruction/moebius1.png", cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread("./Stereo_3D_Reconstruction/moebius2.png", cv2.IMREAD_GRAYSCALE)
    img_left_color = cv2.cvtColor(cv2.imread("./Stereo_3D_Reconstruction/moebius1.png"), cv2.COLOR_BGR2RGB) # BGR
    point_cloud_cluster = 100
    focal_length_1, focal_length_2 = 1512.0, 1512.0
    cx, cy = 695, 555
    NNDR_THRESHOLD = 0.77
    
elif IMAGE_PAIR =="ROCKS":
    img_left = cv2.imread("./Stereo_3D_Reconstruction/rocks1.png", cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread("./Stereo_3D_Reconstruction/rocks2.png", cv2.IMREAD_GRAYSCALE)
    img_left_color = cv2.cvtColor(cv2.imread("./Stereo_3D_Reconstruction/rocks1.png"), cv2.COLOR_BGR2RGB) # BGR
    point_cloud_cluster = 30
    focal_length_1, focal_length_2 = 1512.0, 1512.0
    cx, cy = 638, 555
    NNDR_THRESHOLD = 0.9
    
elif IMAGE_PAIR =="CLOTH":
    img_left = cv2.imread("./Stereo_3D_Reconstruction/cloth1.png", cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread("./Stereo_3D_Reconstruction/cloth2.png", cv2.IMREAD_GRAYSCALE)
    img_left_color = cv2.cvtColor(cv2.imread("./Stereo_3D_Reconstruction/cloth1.png"), cv2.COLOR_BGR2RGB) # BGR
    point_cloud_cluster = 300
    focal_length_1, focal_length_2 = 1050.0, 1050.0
    cx, cy = 641, 555
    NNDR_THRESHOLD = 0.9
    
########################################### Camera Intrinsic Matrix
K = np.array([[focal_length_1, 0, cx],
              [0, focal_length_2, cy],
              [0, 0, 1]])

In [3]:
########################################### Select SIFT_THRESHOLD

SIFT_THRESHOLD = "EXTREMELY_LOW"     ### Choices:     EXTREMELY_LOW     NORMAL
if SIFT_THRESHOLD == "NORMAL":
    sift = cv2.SIFT_create() # Standard SIFT Thresholds
elif SIFT_THRESHOLD == "EXTREMELY_LOW":
    sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=4, contrastThreshold=1e-9, edgeThreshold=100, sigma=0.7) # Extremely Low SIFT Thresholds

kp1, des1 = sift.detectAndCompute(img_left, None)
kp2, des2 = sift.detectAndCompute(img_right, None)

########################################### Feature Matching (FLANN)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=10)
search_params = dict(checks=100)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

### NNDR based Matching (Lowe's Ratio Test)
good_matches = []
pts1 = []
pts2 = []

for m, n in matches:
    if m.distance < NNDR_THRESHOLD * n.distance:
        good_matches.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)
pts1 = np.array(pts1)
pts2 = np.array(pts2)
print(f"Quantity of Matches: {len(good_matches)}")

Quantity of Matches: 19742


In [4]:
########################################### Estimate Essential Matrix
E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.99999, threshold=0.5, maxIters=100000)

pts1 = pts1[mask.ravel() == 1]
pts2 = pts2[mask.ravel() == 1]

print(f"Good Matches after Outlier Rejection: {pts1.shape[0]}")

### Recovering Camera Pose
_, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)

### Projection matrices
P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = K @ np.hstack((R, t))

### Triangulation
pts1_h = pts1.T
pts2_h = pts2.T

points_4d = cv2.triangulatePoints(P1, P2, pts1_h, pts2_h)
points_3d = points_4d[:3] / points_4d[3]
points_3d = points_3d.T

Good Matches after Outlier Rejection: 14128


### **Colored 3D Reconstruction**

In [5]:
### Function for Projecting Points
def project_points(points_3d, K):
    points_h = np.hstack((points_3d, np.ones((points_3d.shape[0], 1))))
    proj = (K @ points_h[:, :3].T).T
    proj = proj[:, :2] / proj[:, 2:3]
    return proj

In [6]:
h, w, _ = img_left_color.shape
projected_pts = project_points(points_3d, K)
colors, valid_colors, valid_points = [], [], []

for i, (u, v) in enumerate(projected_pts):
    u, v = int(round(u)), int(round(v))
    if 0 <= u < w and 0 <= v < h:
        color = img_left_color[v, u] / 255.0  # normalize
        valid_points.append(points_3d[i])
        valid_colors.append(color)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(valid_points))
pcd.colors = o3d.utility.Vector3dVector(np.array(valid_colors))

### Remove Outliers
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=point_cloud_cluster, std_ratio=0.5)

### Front View
o3d.visualization.draw_geometries([pcd], zoom=0.1, front=[0.0, 0.0, -1.0], lookat=[0.0, 0.0, 0.0], up=[0.0, -1.0, 0.0])

### Create Visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(visible=True, width=1280, height=720)
vis.add_geometry(pcd)

# Access Camera and Set Camera Parameters
ctr = vis.get_view_control()
ctr.set_zoom(0.1)
ctr.set_front([0.0, 0.0, -1.0])
ctr.set_lookat([0.0, 0.0, 0.0])
ctr.set_up([0.0, -1.0, 0.0])
vis.poll_events()
vis.update_renderer()

### Save 2D Snapshot
vis.capture_screen_image("point_cloud_SIFT.png")
vis.destroy_window() # Cleanup